# Load modules and Structure files

In [1]:
# Python script to calculate oligomerization state for each monomer chain and inter-contact map of protein chains residues

import numpy as np
import copy as cp
import subprocess as sp
import os as os
import shutil as sh
import MDAnalysis as mdana
import sys
from MDAnalysis.analysis.distances import distance_array
import networkx as nx
import pandas as pd
import mdtraj as md
import matplotlib
matplotlib.use("TkAgg")
%matplotlib nbagg
from matplotlib import pyplot as plt

MolDir='/u2/data/suman/glycans/AA/gags-ab16-22/gags-abeta16_22-5us/dimer/h6s/'
Dir = MolDir
os.chdir(Dir)

#input parameters

ref_structure='pg_ff1.pdb'
traj='pg_f.xtc'
Min_Distance=4
hetero_mols=2
chains_per_mol=2
cc=hetero_mols**chains_per_mol

# Create groups as per requirement

In [2]:
#structure parameters

topology = md.load(ref_structure).topology
trajectory = md.load(traj, top=ref_structure)
frames=trajectory.n_frames              #No. of frames
chains=topology.n_chains                #No. of chains
#atoms=int(topology.n_atoms/chains)         #No. of atoms in each monomer 
#AminoAcids = int(topology.n_residues/chains)-2     #No. of res per protein chain ('-2' avoid the N- and C- cap res as individual res)

for ch in range(0,cc):
    globals()["type_"+ str(ch)+ "_atoms"]=topology.chain(ch).n_atoms            #No. of atoms in each  chain
    globals()["type_"+ str(ch)+ "_residues"]=int(topology.chain(ch).n_residues) #No. of res per  chain
    #This is only valid for this example and can be avoided.
    #No. of res per protein chain ('-2' avoid the N- and C- cap res as individual res)
#type_2_residues=int(topology.chain(2).n_residues)-2 
#type_3_residues=int(topology.chain(3).n_residues)-2

#This is only valid for this example and can be avoided.
#No. of res per protein chain ('-2' avoid the N- and C- cap res as individual res)
for chp in range(int(cc/chains_per_mol),cc):
    globals()["type_"+ str(chp)+ "_residues"]=int(topology.chain(chp).n_residues)-2

for chn in range(0,cc):
    globals()["type_"+ str(chn)+ "_isum"]=1
    globals()["type_"+ str(chn)+ "_atoms_list"]=[]
    globals()["type_"+ str(chn)+ "_atomsperAminoAcid"]=[]
    globals()["type_"+ str(chn)+ "_residue_list"]=[]
    #isum=1
    #atoms_list=[]
    #atomsperAminoAcid=[]
    #residue_list=[]

    for residue in topology.chain(chn).residues:
        globals()["type_"+ str(chn)+ "_atoms_list"].append(residue.n_atoms)
        globals()["type_"+ str(chn)+ "_residue_list"].append(residue)
        ', '.join(map(lambda x: "'" + x + "'", str(globals()["type_"+ str(chn)+ "_residue_list"])))
    #The N- and C- cap res's are part of the 1st and last res index. If no N- and C- cap res's for the protein, comment the line below using "#"

    for ii in range(len(globals()["type_"+ str(chn)+ "_atoms_list"])):
        globals()["type_"+ str(chn)+ "_isum"] = globals()["type_"+ str(chn)+ "_isum"] + globals()["type_"+ str(chn)+ "_atoms_list"][ii]
        globals()["type_"+ str(chn)+ "_atomsperAminoAcid"].append(globals()["type_"+ str(chn)+ "_isum"])
    globals()["type_"+ str(chn)+ "_atomsperAminoAcid"].insert(0, 1)
#type_2_atomsperAminoAcid.insert(0, 1); type_3_atomsperAminoAcid.insert(0, 1)
    #del globals()["type_"+ str(chn)+ "_atomsperAminoAcid"][1]; del globals()["type_"+ str(chn)+ "_atomsperAminoAcid"][-2]      
#The N- and C- cap res's are part of the 1st and last res index. 
#If no N- and C- cap res's for the protein, comment the line below using "#"

#del type_2_atomsperAminoAcid[1];del type_2_atomsperAminoAcid[-2]
#del type_3_atomsperAminoAcid[1];del type_3_atomsperAminoAcid[-2]
#del type_2_residue_list[0]; del type_2_residue_list[-1]
#del type_3_residue_list[0]; del type_3_residue_list[-1]

#The N- and C- cap res's are part of the 1st and last res index. 
#If no N- and C- cap res's for the protein, comment the line below using "#"

for chp in range(int(cc/chains_per_mol),cc):
    del globals()["type_"+ str(chp)+ "_atomsperAminoAcid"][1]
    del globals()["type_"+ str(chp)+ "_atomsperAminoAcid"][-2]
    del globals()["type_"+ str(chp)+ "_residue_list"][0]
    del globals()["type_"+ str(chp)+ "_residue_list"][-1]


In [3]:
type_0_residues, type_1_residues,type_2_residues,type_3_residues

(10, 10, 7, 7)

In [4]:
type_0_residues, type_1_residues,type_2_residues,type_3_residues

(10, 10, 7, 7)

In [5]:
type_0_residue_list, type_1_residue_list, type_2_residue_list, type_3_residue_list

([AGL1, GU2, AGL3, GU4, AGL5, GU6, AGL7, GU8, AGL9, GU10],
 [AGL1, GU2, AGL3, GU4, AGL5, GU6, AGL7, GU8, AGL9, GU10],
 [LYS16, LEU17, VAL18, PHE19, PHE20, ALA21, GLU22],
 [LYS16, LEU17, VAL18, PHE19, PHE20, ALA21, GLU22])

In [6]:
type_0_residue_list, type_1_residue_list, type_2_residue_list, type_3_residue_list

([AGL1, GU2, AGL3, GU4, AGL5, GU6, AGL7, GU8, AGL9, GU10],
 [AGL1, GU2, AGL3, GU4, AGL5, GU6, AGL7, GU8, AGL9, GU10],
 [LYS16, LEU17, VAL18, PHE19, PHE20, ALA21, GLU22],
 [LYS16, LEU17, VAL18, PHE19, PHE20, ALA21, GLU22])

In [7]:
type_0_atomsperAminoAcid, type_1_atomsperAminoAcid, type_2_atomsperAminoAcid, type_3_atomsperAminoAcid

([1, 33, 52, 82, 101, 131, 150, 180, 199, 229, 249],
 [1, 33, 52, 82, 101, 131, 150, 180, 199, 229, 249],
 [1, 29, 48, 64, 84, 104, 114, 135],
 [1, 29, 48, 64, 84, 104, 114, 135])

In [8]:
type_0_atomsperAminoAcid, type_1_atomsperAminoAcid, type_2_atomsperAminoAcid, type_3_atomsperAminoAcid

([1, 33, 52, 82, 101, 131, 150, 180, 199, 229, 249],
 [1, 33, 52, 82, 101, 131, 150, 180, 199, 229, 249],
 [1, 29, 48, 64, 84, 104, 114, 135],
 [1, 29, 48, 64, 84, 104, 114, 135])

In [9]:
type_0_atoms_list, type_1_atoms_list, type_2_atoms_list, type_3_atoms_list

([32, 19, 30, 19, 30, 19, 30, 19, 30, 20],
 [32, 19, 30, 19, 30, 19, 30, 19, 30, 20],
 [6, 22, 19, 16, 20, 20, 10, 15, 6],
 [6, 22, 19, 16, 20, 20, 10, 15, 6])

In [10]:
type_0_atoms_list, type_1_atoms_list, type_2_atoms_list, type_3_atoms_list

([32, 19, 30, 19, 30, 19, 30, 19, 30, 20],
 [32, 19, 30, 19, 30, 19, 30, 19, 30, 20],
 [6, 22, 19, 16, 20, 20, 10, 15, 6],
 [6, 22, 19, 16, 20, 20, 10, 15, 6])

# Protein-Ligand Analysis

In [11]:
# Create Universe

uni = mdana.Universe(ref_structure,traj)
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]



m_start=0
atom_Groups= [[] for x in range(cc)]

p= (globals()["type_"+ str(0)+ "_atoms"]- globals()["type_"+ str(2)+ "_atoms"])*chains_per_mol


for m in range(0,int(cc/chains_per_mol)):
    m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1)
    atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
    m_start = m_end + 1
    
for m in range(int(cc/chains_per_mol),cc):
    m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1) + p
    atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])  
    m_start = m_end + 1

fileout1 =  open('n-contact-groups.dat','w')
fileout2 =  open('n-contact-states.dat','w')

for tt in uni.trajectory[0:]:
    fileout1.write (str(tt.frame) + '\t')
    fileout2.write (str(tt.frame) + '\t')
    
    mySet = set([])
    graph = []
    
    
    m_start=0
    atom_Groups= [[] for x in range(cc)]
    p= (globals()["type_"+ str(0)+ "_atoms"]- globals()["type_"+ str(2)+ "_atoms"])*chains_per_mol

    for m in range(0,int(cc/chains_per_mol)):
        m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1)
        atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
        m_start = m_end + 1
    
    for m in range(int(cc/chains_per_mol),cc):
        m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1) + p
        atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])  
        m_start = m_end + 1
        
    count = 0
    #make changes below according to requirement.
    for i in range(cc-1):
        for j in range(i+1,cc):
            dist = distance_array(atom_Groups[i][0].positions,atom_Groups[j][0].positions,box).min()
            if(dist <= Min_Distance):
                my_tuple = (i,j)
                mySet.add(my_tuple)
                
    graph = nx.from_edgelist(mySet)   
    for i in range(cc):
        if i not in list(graph):
            fileout1.write ('['+ str(i)+']' + '\t')
            fileout2.write ('1' + '\t')
        else:
            fileout1.write (str(list(nx.node_connected_component(graph, i))) + '\t')
            fileout2.write (str(len(list(nx.node_connected_component(graph, i)))) + '\t')
    fileout1.write ('\n')
    fileout2.write ('\n')
   
    
fileout1.close()
fileout2.close()

In [12]:
atom_Groups[0], atom_Groups[1], atom_Groups[2], atom_Groups[3]

([<AtomGroup with 248 atoms>],
 [<AtomGroup with 248 atoms>],
 [<AtomGroup with 134 atoms>],
 [<AtomGroup with 134 atoms>])

In [13]:
df=pd.read_table("n-contact-groups.dat", usecols=[1,2,3,4])

/home/suman/anaconda3/envs/pyemma_tutorials/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [14]:
df.tail()

,[0],[1],[2],[3]
124995,[0],[1],"[2, 3]","[2, 3]"
124996,[0],[1],"[2, 3]","[2, 3]"
124997,[0],[1],"[2, 3]","[2, 3]"
124998,[0],[1],"[2, 3]","[2, 3]"
124999,[0],[1],"[2, 3]","[2, 3]"


# Build ContactMap for Protein-Ligand, Ligand-only, Protein-only systems

In [15]:
# Get oligomerization data

OligoStates = [[0 for z in range(cc)] for x in range(frames+1)]
file = open("n-contact-groups.dat",'r')
line = file.readline()
j = 0
while line:
    temp = line.split('\t')
    for k in range(cc):
        OligoStates[j][k] = temp[k + 1][1:-1].split(',')
    j += 1
    line = file.readline()
file.close


# Create contact matrix
#make changes to dimensions based on the hetero atomtypes e.g.type_0_residues or type_2_residues
ContactMap_pg = [[0 for x in range(type_2_residues)] for y in range(type_0_residues)]
ContactMap_g = [[0 for x in range(type_0_residues)] for y in range(type_0_residues)]
ContactMap_p = [[0 for x in range(type_2_residues)] for y in range(type_2_residues)]
# Create atom groups for each amino acid of each monomer
AtomGroups= [[] for x in range(cc)]

for m in range(0,cc):
    for aa in range(0,globals()["type_"+ str(m)+ "_residues"]):
        AtomGroups[m].extend([uni.select_atoms('bynum '+str(globals()["type_"+str(m)+"_atoms"]*m+globals()["type_"+str(m)+"_atomsperAminoAcid"][aa])+':'+str(globals()["type_"+str(m)+"_atoms"]*m+globals()["type_"+str(m)+"_atomsperAminoAcid"][aa+1]-1))])

        
count_pg = 0
count_g = 0
count_p = 0
for n,t in enumerate(uni.trajectory[1:]):
    on = 0
    Groups = []
    for i in OligoStates[n]:
        if len(i) > 1:
            on = 1
            Groups.extend([i])
    Set = set(tuple(x) for x in Groups)
    Groups = [ list(x) for x in Set ]
    
    if on == 1:
    # Calculate dimension of the box to considered PBC
        box = t.dimensions[:6]
        for g in Groups:
        # Calculate contacts
            for n1,i in enumerate(g):
                for j in g[(n1 + 1)::]:
                    if len(AtomGroups[int(float(i))]) != len(AtomGroups[int(float(j))]):
                        count_pg += 1
                        for n2,atoms1 in enumerate(AtomGroups[int(float(i))]):
                            for n3,atoms2 in enumerate(AtomGroups[int(float(j))]):
                                if ((distance_array(atoms1.positions,atoms2.positions,box).min()) <= Min_Distance):
                                    ContactMap_pg[n2][n3] +=1
                                    #ContactMap[n3][n2] +=1
                    elif ((len(AtomGroups[int(float(i))]) + len(AtomGroups[int(float(j))])) == type_0_residues*chains_per_mol) :
                        count_g += 1
                        for n2,atoms1 in enumerate(AtomGroups[int(float(i))]):
                            for n3,atoms2 in enumerate(AtomGroups[int(float(j))]):
                                if ((distance_array(atoms1.positions,atoms2.positions,box).min()) <= Min_Distance):
                                    ContactMap_g[n2][n3] +=1
                                    ContactMap_g[n3][n2] +=1                
                    elif ((len(AtomGroups[int(float(i))]) + len(AtomGroups[int(float(j))])) == type_2_residues*chains_per_mol) :
                        count_p += 1
                        for n2,atoms1 in enumerate(AtomGroups[int(float(i))]):
                            for n3,atoms2 in enumerate(AtomGroups[int(float(j))]):
                                if ((distance_array(atoms1.positions,atoms2.positions,box).min()) <= Min_Distance):
                                    ContactMap_p[n2][n3] +=1
                                    ContactMap_p[n3][n2] +=1
                                    
print(count_pg)
print(count_g)
print(count_p)
Norm_ContactMap_pg = np.true_divide(ContactMap_pg,float(count_pg))
Norm_ContactMap_g = np.true_divide(ContactMap_g,float(count_g)*2.0)
Norm_ContactMap_p = np.true_divide(ContactMap_p,float(count_p)*2.0)
# Save contact map in a file

fileout = open ('n-contact-map-pg.dat','w')
for i in Norm_ContactMap_pg:
	for j in i:
		fileout.write ("%.5f\t" %(float(j)))
	fileout.write ('\n')
fileout.close()

fileout = open ('n-contact-map-g.dat','w')
for i in Norm_ContactMap_g:
	for j in i:
		fileout.write ("%.5f\t" %(float(j)))
	fileout.write ('\n')
fileout.close()

fileout = open ('n-contact-map-p.dat','w')
for i in Norm_ContactMap_p:
	for j in i:
		fileout.write ("%.5f\t" %(float(j)))
	fileout.write ('\n')
fileout.close()

77461
2376
60876


In [16]:
AtomGroups[0], AtomGroups[1], AtomGroups[2], AtomGroups[3]

([<AtomGroup with 32 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 20 atoms>],
 [<AtomGroup with 32 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 30 atoms>,
  <AtomGroup with 20 atoms>],
 [<AtomGroup with 28 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 16 atoms>,
  <AtomGroup with 20 atoms>,
  <AtomGroup with 20 atoms>,
  <AtomGroup with 10 atoms>,
  <AtomGroup with 21 atoms>],
 [<AtomGroup with 28 atoms>,
  <AtomGroup with 19 atoms>,
  <AtomGroup with 16 atoms>,
  <AtomGroup with 20 atoms>,
  <AtomGroup with 20 atoms>,
  <AtomGroup with 10 atoms>,
  <AtomGroup with 21 atoms>])

In [17]:
len(AtomGroups[int(float(0))]), len(AtomGroups[int(float(1))]), len(AtomGroups[int(float(2))]), len(AtomGroups[int(float(3))])

(10, 10, 7, 7)

# Protein-only Analysis

In [18]:
type_2_atoms, type_3_atoms

(134, 134)

In [19]:
# contact map states for protein part

uni = mdana.Universe(ref_structure,traj)
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]



m_start=(type_0_atoms*chains_per_mol) +1
pro_atom_Groups= [[] for x in range(int(cc/chains_per_mol))]

    
for m in range(int(cc/chains_per_mol)):
    m_end = (type_0_atoms*chains_per_mol) + globals()["type_"+ str(m+2)+ "_atoms"] * (m+1)
    print(m_end)
    pro_atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])  
    m_start = m_end + 1

fileout1 =  open('n-pro-contact-groups.dat','w')
fileout2 =  open('n-pro-contact-states.dat','w')

for tt in uni.trajectory[0:]:
    fileout1.write (str(tt.frame) + '\t')
    fileout2.write (str(tt.frame) + '\t')
    
    mySet = set([])
    graph = []
    
    
    m_start=(type_0_atoms*chains_per_mol) +1
    pro_atom_Groups= [[] for x in range(int(cc/chains_per_mol))]

    for m in range(int(cc/chains_per_mol)):
        m_end = (type_0_atoms*chains_per_mol) + globals()["type_"+ str(m+2)+ "_atoms"] * (m+1)
        pro_atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])  
        m_start = m_end + 1
        
    count = 0
    #make changes below according to requirement.
    for i in range(int(cc/chains_per_mol)-1):
        for j in range(i+1,int(cc/chains_per_mol)):
            dist = distance_array(pro_atom_Groups[i][0].positions,pro_atom_Groups[j][0].positions,box).min()
            if(dist <= Min_Distance):
                my_tuple = (i+2,j+2)
                mySet.add(my_tuple)
                
    graph = nx.from_edgelist(mySet)   
    for i in range(int(cc/chains_per_mol), cc):
        if i not in list(graph):
            fileout1.write ('['+ str(i)+']' + '\t')
            fileout2.write ('1' + '\t')
        else:
            fileout1.write (str(list(nx.node_connected_component(graph, i))) + '\t')
            fileout2.write (str(len(list(nx.node_connected_component(graph, i)))) + '\t')
    fileout1.write ('\n')
    fileout2.write ('\n')
   
    
fileout1.close()
fileout2.close()

630
764


In [20]:
pro_atom_Groups[0], pro_atom_Groups[1]

([<AtomGroup with 134 atoms>], [<AtomGroup with 134 atoms>])

In [21]:
#Highest Oligomer size in each frame

states=open('n-pro-contact-states.dat', 'r')
ter=states.readlines()[0:frames+1]

result=[]
for freq in (ter):
    	result.append([int(hist) for hist in freq.strip().split('\t')[1:]])

fileout3 = open ('n-pro-oligo-highest-size.dat', 'w')
for oli_count in range(len(ter)):
	fileout3.write("{} {} {}\n".format(oli_count, '\t', np.max(result[oli_count])))
fileout3.close()



# Block Average


size_data = np.loadtxt('n-pro-contact-states.dat')
window = 20                                         	# specify over how many frames the running average is to be calculated
weights = np.repeat(1.0,window)/window
size_data_m = np.convolve(size_data[:,1],weights,'valid')

fileout4 = open('n-pro-oligo-block-average.dat', 'w')
for t,b in enumerate(size_data_m):
    	fileout4.write("{} {} {}\n".format(t, '\t', b))
fileout4.close()

# Ligand-only Analysis

In [22]:
# Create Universe

uni = mdana.Universe(ref_structure,traj)
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]



m_start=0
lig_atom_Groups= [[] for x in range(int(cc/chains_per_mol))]

for m in range(0,int(cc/chains_per_mol)):
    m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1)
    lig_atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
    m_start = m_end + 1
    

fileout1 =  open('n-lig-contact-groups.dat','w')
fileout2 =  open('n-lig-contact-states.dat','w')

for tt in uni.trajectory[0:]:
    fileout1.write (str(tt.frame) + '\t')
    fileout2.write (str(tt.frame) + '\t')
    
    mySet = set([])
    graph = []
    
    
    m_start=0
    lig_atom_Groups= [[] for x in range(int(cc/chains_per_mol))]
   
    for m in range(0,int(cc/chains_per_mol)):
        m_end = globals()["type_"+ str(m)+ "_atoms"] * (m+1)
        lig_atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
        m_start = m_end + 1

        
    count = 0
    #make changes below according to requirement.
    for i in range(int(cc/chains_per_mol)-1):
        for j in range(i+1,int(cc/chains_per_mol)):
            dist = distance_array(lig_atom_Groups[i][0].positions,lig_atom_Groups[j][0].positions,box).min()
            if(dist <= Min_Distance):
                my_tuple = (i,j)
                mySet.add(my_tuple)
                
    graph = nx.from_edgelist(mySet)   
    for i in range(int(cc/chains_per_mol)):
        if i not in list(graph):
            fileout1.write ('['+ str(i)+']' + '\t')
            fileout2.write ('1' + '\t')
        else:
            fileout1.write (str(list(nx.node_connected_component(graph, i))) + '\t')
            fileout2.write (str(len(list(nx.node_connected_component(graph, i)))) + '\t')
    fileout1.write ('\n')
    fileout2.write ('\n')
   
    
fileout1.close()
fileout2.close()

In [23]:
#Highest Oligomer size in each frame

states=open('n-lig-contact-states.dat', 'r')
ter=states.readlines()[0:frames+1]

result=[]
for freq in (ter):
    	result.append([int(hist) for hist in freq.strip().split('\t')[1:]])

fileout3 = open ('n-lig-oligo-highest-size.dat', 'w')
for oli_count in range(len(ter)):
	fileout3.write("{} {} {}\n".format(oli_count, '\t', np.max(result[oli_count])))
fileout3.close()



# Block Average


size_data = np.loadtxt('n-lig-contact-states.dat')
window = 20                                         	# specify over how many frames the running average is to be calculated
weights = np.repeat(1.0,window)/window
size_data_m = np.convolve(size_data[:,1],weights,'valid')

fileout4 = open('n-lig-oligo-block-average.dat', 'w')
for t,b in enumerate(size_data_m):
    	fileout4.write("{} {} {}\n".format(t, '\t', b))
fileout4.close()